In [ ]:
import torch, platform
print("Python:", platform.python_version())
print("CUDA available:", torch.cuda.is_available())
print("Device:", torch.device("cuda" if torch.cuda.is_available() else "cpu"))


Python: 3.12.11
CUDA available: False
Device: cpu


In [ ]:
!pip -q install -U "transformers>=4.56.1" "datasets>=2.20.0" "peft>=0.17.1" rank-bm25 sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 20.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.


In [ ]:

ADAPTER_ZIP = "/content/biobart_adapter.zip" # <- edit path


In [ ]:
!unzip -o "$ADAPTER_ZIP" -d /content/
ADAPTER_DIR = "/content/biobart_adapter.zip"  # <- adjust if your folder name differs after unzip


Archive:  /content/biobart_adapter.zip
  inflating: /content/special_tokens_map.json  
  inflating: /content/tokenizer.json  
  inflating: /content/merges.txt     
  inflating: /content/training_args.bin  
  inflating: /content/tokenizer_config.json  
  inflating: /content/adapter_config.json  
  inflating: /content/README.md      
  inflating: /content/vocab.json     
  inflating: /content/adapter_model.safetensors  


In [ ]:
import os, shutil, pathlib

SRC = pathlib.Path("/content")
ADAPTER_DIR = "/content/biobart_adapter"   # new folder
os.makedirs(ADAPTER_DIR, exist_ok=True)

for name in ["adapter_config.json", "adapter_model.safetensors", "adapter_model.bin"]:
    p = SRC / name
    if p.exists():
        shutil.move(str(p), ADAPTER_DIR)

!ls -la "$ADAPTER_DIR"


total 1752
drwxr-xr-x 2 root root    4096 Sep 19 08:04 .
drwxr-xr-x 1 root root    4096 Sep 19 08:04 ..
-rw-r--r-- 1 root root     854 Sep 12 15:25 adapter_config.json
-rw-r--r-- 1 root root 1779432 Sep 12 15:25 adapter_model.safetensors


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import PeftModel

BASE_MODEL = "GanjinZero/biobart-base"   # use the same base you trained on
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
base = AutoModelForSeq2SeqLM.from_pretrained(BASE_MODEL)
model = PeftModel.from_pretrained(base, ADAPTER_DIR).eval().to(device)

print("Loaded adapters from:", ADAPTER_DIR, "| device:", device)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/279M [00:00<?, ?B/s]

Loaded adapters from: /content/biobart_adapter | device: cpu


In [ ]:
DATA_PATH = "/content/merged_corpus_cleaned_Final.jsonl"

In [ ]:
import re, numpy as np
from datasets import load_dataset
from rank_bm25 import BM25Okapi

def _norm(s: str) -> str:
    return re.sub(r"\s+"," ", (s or "").strip()) if s else ""

def _tok(s: str):
    return re.findall(r"[A-Za-z0-9][A-Za-z0-9\-]+", (s or "").lower())

def _clean_answer_text(a: str) -> str:
    s = _norm(a)
    s = re.sub(r"(?is)\bref(?:erences?)?:.*$", "", s)
    s = re.sub(r"style=\".*?\"", "", s)
    s = re.sub(r"\s{2,}", " ", s)
    parts = re.split(r'(?<=[.!?])\s+', s)
    s = " ".join(parts[:2]) if parts else s
    if len(s) > 400: s = s[:400].rsplit(" ",1)[0]
    return s.strip()

raw = load_dataset("json", data_files={"train": DATA_PATH})["train"]
raw = raw.filter(lambda ex: bool(ex.get("answer")))

CTX, CID, seen = [], [], set()
for a, i in zip(raw["answer"], raw["id"]):
    ca = _clean_answer_text(a)
    if ca and ca not in seen:
        seen.add(ca); CTX.append(ca); CID.append(i)

DOCS_TOK = [_tok(t) for t in CTX]
bm25 = BM25Okapi(DOCS_TOK)
print(f"BM25 ready | contexts: {len(CTX)}")


Generating train split: 0 examples [00:00, ? examples/s]

Filter:   0%|          | 0/183030 [00:00<?, ? examples/s]

BM25 ready | contexts: 169643


In [ ]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

def retrieve_candidates(question: str, k_bm25=400, k_dense=80, k_final=6):
    qtok = _tok(question)
    scores_bm25 = bm25.get_scores(qtok)
    if k_bm25 < len(scores_bm25):
        idx = np.argpartition(-scores_bm25, k_bm25)[:k_bm25]
        idx = idx[np.argsort(-scores_bm25[idx])]
    else:
        idx = np.argsort(-scores_bm25)

    cand_ctx = [CTX[i] for i in idx]
    q_emb    = embedder.encode([_norm(question)], normalize_embeddings=True)
    c_embs   = embedder.encode(cand_ctx, batch_size=128, normalize_embeddings=True)
    dense_scores = (c_embs @ q_emb.T).ravel()

    order = np.argsort(-dense_scores)[:min(k_dense, len(dense_scores))]
    idx   = idx[order]
    cand_ctx = [CTX[i] for i in idx]

    order2 = np.argsort(-dense_scores[order])[:min(k_final, len(order))]
    idx    = idx[order2]
    cand_ctx = [CTX[i] for i in idx]
    scores   = dense_scores[order][order2]

    hits = [(CID[i], cand_ctx[j], float(scores[j])) for j, i in enumerate(idx)]
    return hits


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# ===================== RETRIEVAL + CLEAN QA (CPU-friendly) =====================
# Run AFTER you load your (fine-tuned) BioBART `model` and `tokenizer`.
# Only dependency: internet for small models on first run.

import sys, subprocess, importlib, re, math, time, os, gc
import numpy as np, torch

device = "cuda" if torch.cuda.is_available() else "cpu"
assert "model" in globals() and "tokenizer" in globals(), "Load your model+tokenizer first."

# ---------- Quiet installers ----------
def _pip(spec): subprocess.run([sys.executable, "-m", "pip", "install", "-qU", spec], check=True)
def ensure(spec, module):
    try: return importlib.import_module(module)
    except Exception: _pip(spec); return importlib.import_module(module)

ensure("packaging", "packaging")
datasets = ensure("datasets>=2.20.0", "datasets")
rankbm25 = ensure("rank-bm25", "rank_bm25")
senttx   = ensure("sentence-transformers>=2.7.0", "sentence_transformers")  # CrossEncoder needs >=2.7
from datasets import load_dataset
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer, CrossEncoder

# ---------- Config: EDIT PATH ----------
DATA_PATH = "/content/merged_corpus_cleaned_Final.jsonl"  # set to your repaired JSONL file

# Retrieval sizes (tune for speed/quality)
TOP_BM25   = 400   # lexical shortlist
TOP_DENSE  = 60    # MiniLM rerank keep
TOP_FINAL  = 10    # cross-encoder rerank keep
TOP_CONTEXT = 2    # how many sentences finally feed to the generator

# Optional entailment (keeps answers honest; small cost)
USE_ENTAILMENT = True
ENTAIL_THRESH  = 0.62  # probability of entailment required

# ---------- Make BART decode robust (avoid empty/echo) ----------
if getattr(model.config, "decoder_start_token_id", None) is None:
    model.config.decoder_start_token_id = (
        getattr(tokenizer, "bos_token_id", None)
        or getattr(tokenizer, "cls_token_id", None)
        or getattr(tokenizer, "eos_token_id", None)
        or 0
    )
if getattr(tokenizer, "pad_token_id", None) is None:
    tokenizer.pad_token = tokenizer.eos_token or tokenizer.sep_token or tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)

# ---------- Light text helpers ----------
STOP = set("a an the and or of to for with without on in at by from as is are was were be been being than that this these those what which who whom whose when where why how does do did will would should could can may might has have had into".split())
def _norm(s): return re.sub(r"\s+", " ", (s or "").strip())
def _tok(s):  return re.findall(r"[A-Za-z0-9][A-Za-z0-9\-]+", (s or "").lower())
def _overlap_ratio(q, c):
    qset = set(t for t in _tok(q) if t not in STOP)
    cset = set(t for t in _tok(c) if t not in STOP)
    return len(qset & cset) / max(1, len(qset))

# Intent detection
def _is_polar(q: str) -> bool:
    q = (q or "").strip().lower()
    starts = tuple("do does did is are am was were can could should would will has have had may might".split())
    verbs  = ("contain","include","cause","contraindicated","safe","allowed","recommended",
              "effective","increase","decrease","prevent","indicated","contraindicated")
    return q.endswith("?") and (q.split()[0] in starts or any(v in q for v in verbs))
def _is_compare(q: str):
    s = (q or "").lower()
    m = re.search(r"(?:difference between|compare)\s+(.+?)\s+(?:and|vs\.?|versus)\s+(.+?)\??$", s)
    if m: return True, (m.group(1).strip(), m.group(2).strip())
    m = re.search(r"(.+?)\s+(?:vs\.?|versus)\s+(.+?)\??$", s)
    if m: return True, (m.group(1).strip(), m.group(2).strip())
    return False, None

# Clean an answer blob (drop refs, html noise)
def _clean_answer_text(a: str) -> str:
    s = _norm(a)
    s = re.sub(r"(?is)\bref(?:erences?)?:.*$", "", s)
    s = re.sub(r"style=\".*?\"", "", s)
    s = re.sub(r"\s{2,}", " ", s)
    return s.strip()

# Simple sentence splitter (regex; no heavy punkt download)
_SENT_SPLIT = re.compile(r'(?<=[\.\!\?])\s+(?=[A-Z0-9(])')
def _split_sentences(text: str):
    text = _norm(text)
    if not text: return []
    sents = _SENT_SPLIT.split(text)
    # keep mid-length factual sentences
    out = []
    for s in sents:
        ss = s.strip().strip("•-–—")
        if 24 <= len(ss) <= 300:
            out.append(ss)
    return out[:8]  # cap per record to keep index compact

# ---------- Build the sentence index from your dataset ----------
def build_index(data_path=DATA_PATH, verbose=True):
    raw = load_dataset("json", data_files={"train": data_path})["train"]
    if "answer" not in raw.column_names:
        raise ValueError("Expected an 'answer' column in the JSONL.")
    # filter rows with any answer text
    raw = raw.filter(lambda ex: bool(ex.get("answer")))
    CTX_SENTS, META = [], []  # sentences and (orig_id, sent_idx)
    for idx, (a, _id) in enumerate(zip(raw["answer"], raw["id"])):
        clean = _clean_answer_text(a)
        sents = _split_sentences(clean)
        for j, s in enumerate(sents):
            CTX_SENTS.append(s)
            META.append((_id, j))
    if verbose:
        print(f"Indexed sentences: {len(CTX_SENTS)} from {len(raw)} rows")

    # BM25 over tokens
    DOCS_TOK = [_tok(t) for t in CTX_SENTS]
    bm25 = BM25Okapi(DOCS_TOK)

    # Dense embedder + cross-encoder reranker + (optional) NLI entailment
    embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")     # 384-dim
    reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")             # top-10 precise
    nli = None
    if USE_ENTAILMENT:
        nli = CrossEncoder("cross-encoder/nli-deberta-v3-small")                # entailment/neutral/contradiction

    return {"sents": CTX_SENTS, "meta": META, "bm25": bm25,
            "embedder": embedder, "reranker": reranker, "nli": nli}

# Build once (reuse in session)
INDEX = build_index(DATA_PATH)

# ---------- Retrieval pipeline ----------
def retrieve(question: str, k_bm25=TOP_BM25, k_dense=TOP_DENSE, k_final=TOP_FINAL):
    q = _norm(question)
    bm25 = INDEX["bm25"]; SENTS = INDEX["sents"]; META = INDEX["meta"]
    # 1) BM25 shortlist
    scores = bm25.get_scores(_tok(q))
    if len(scores) == 0: return []
    k_bm25 = min(k_bm25, len(scores))
    idx = np.argpartition(-scores, k_bm25-1)[:k_bm25]
    idx = idx[np.argsort(-scores[idx])]
    cand = [SENTS[i] for i in idx]

    # 2) Dense rerank with MiniLM (encode only shortlist)
    q_emb = INDEX["embedder"].encode([q], normalize_embeddings=True)
    c_emb = INDEX["embedder"].encode(cand, batch_size=256, normalize_embeddings=True)
    cos = (c_emb @ q_emb.T).ravel()
    order = np.argsort(-cos)[:min(k_dense, len(cos))]
    idx   = idx[order]; cand = [cand[j] for j in order]

    # 3) Cross-encoder rerank on top-keep
    pairs = [[q, c] for c in cand]
    ce_scores = INDEX["reranker"].predict(pairs, batch_size=64)
    order2 = np.argsort(-ce_scores)[:min(k_final, len(ce_scores))]
    idx    = idx[order2]; cand = [cand[j] for j in order2]
    final  = [(int(i), SENTS[i], META[i], float(ce_scores[k])) for k, i in enumerate(idx)]
    return final

# ---------- Minimal, clean generation ----------
@torch.no_grad()
def _gen(prompt, max_new=72, beams=4, lp=0.65):
    enc = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    enc = {k: v.to(device) for k, v in enc.items()}
    out = model.generate(
        **enc,
        max_new_tokens=max_new,
        do_sample=False,
        num_beams=beams,
        length_penalty=lp,
        no_repeat_ngram_size=3,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        repetition_penalty=1.08,
    )
    return tokenizer.decode(out[0], skip_special_tokens=True).strip()

def _clean_out(text: str) -> str:
    if not text: return ""
    t = text.replace("\r", " ").strip()
    t = re.sub(r"(?i)\b(?:context|question|answer|using only|evidence)\b[:\s].*", "", t)
    t = re.sub(r"\s+", " ", t).strip(" -–—")
    parts = re.split(r'(?<=[.!?])\s+', t)
    out = " ".join(parts[:2]).strip()
    if not out:
        toks = t.split(); out = " ".join(toks[:28]).strip()
    return out

def _force_yesno(ans: str) -> str | None:
    low = ans.lower().strip()
    if low.startswith("yes") or low.startswith("no"):
        head = "Yes" if low.startswith("yes") else "No"
        tail = ans[len(head):].lstrip(" :—-")
        return f"{head} — {' '.join(tail.split()[:28])}" if tail else head
    m = re.search(r"(?i)\b(yes|no)\b.*", ans)
    if m: return _force_yesno(m.group(0))
    return None

# ---------- Optional entailment guard ----------
def _entailed(answer: str, sents: list[str]) -> bool:
    if not USE_ENTAILMENT or not INDEX["nli"]:
        return True
    # Pair (premise=sent, hypothesis=answer)
    pairs = [[s, answer] for s in sents]
    logits = INDEX["nli"].predict(pairs, batch_size=32)  # returns [contradiction, neutral, entailment]
    probs  = torch.softmax(torch.tensor(logits), dim=-1).numpy()
    ent = probs[:, 2].max()  # entailment prob
    return bool(ent >= ENTAIL_THRESH)

# ---------- Main entry ----------
@torch.no_grad()
def ask(question: str, *, style="short"):
    q = question.strip()
    polar = _is_polar(q)
    is_cmp, cmp_terms = _is_compare(q)

    # decode presets
    style = (style or "short").lower()
    if style == "short": dec = dict(max_new=56, beams=4, lp=0.70)
    elif style == "long": dec = dict(max_new=128, beams=4, lp=0.62)
    else:                 dec = dict(max_new=80, beams=4, lp=0.65)

    # Retrieve
    hits = retrieve(q)
    use_ground = False
    if hits:
        top_sent = hits[0][1]
        # require both dense match (via reranker) implicitly and token overlap
        if _overlap_ratio(q, top_sent) >= 0.22:
            use_ground = True

    # Build minimal prompt (avoid echoes)
    if use_ground:
        ctx_sents = [h[1] for h in hits[:TOP_CONTEXT]]
        if polar:
            prompt = f"{' '.join(ctx_sents)}\n\n{q}\n\nRespond with 'Yes' or 'No' first, then one brief reason."
        elif is_cmp and cmp_terms:
            a,b = cmp_terms
            prompt = f"{' '.join(ctx_sents)}\n\n{q}\n\nIn one concise sentence, clearly contrast {a} and {b} (mention both)."
        else:
            prompt = f"{' '.join(ctx_sents)}\n\n{q}\n\nAnswer in one or two sentences."
        raw = _gen(prompt, **dec)
        ans = _clean_out(raw)
        if polar:
            yn = _force_yesno(ans)
            if yn and _entailed(yn, ctx_sents):
                return yn
        elif ans and len(ans.split()) >= 3 and _entailed(ans, ctx_sents):
            return ans
        # if grounded answer is weak/unentailed, fall back

    # Fallback: model knowledge (never say “insufficient”)
    if polar:
        fb = f"{q}\n\nRespond with 'Yes' or 'No' first, then one brief reason."
        raw = _gen(fb, **dec)
        ans = _clean_out(raw)
        yn = _force_yesno(ans)
        return yn or ( "Best estimate: " + (ans if ans else "No definitive answer.") )
    elif is_cmp and cmp_terms:
        a,b = cmp_terms
        fb = f"{q}\n\nIn one concise sentence, clearly contrast {a} and {b} (mention both)."
        ans = _clean_out(_gen(fb, **dec))
        if not ((a.lower() in ans.lower()) and (b.lower() in ans.lower())):
            ans = f"{a.title()} and {b.title()} differ primarily in {ans}"
        return ans
    else:
        fb = f"{q}\n\nAnswer in one or two sentences."
        ans = _clean_out(_gen(fb, **dec))
        return ans if len(ans.split()) >= 3 else "Best estimate: No definitive answer."

# ------------------ Examples (uncomment to test) ------------------
# print(ask("Do 5 mg Zolmitriptan tablets contain gluten?"))
# print(ask("What are the neurological symptoms of hyperthyroidism?"))
# print(ask("Explain the difference between type 1 and type 2 diabetes.", style="short"))


Filter:   0%|          | 0/183030 [00:00<?, ? examples/s]

Indexed sentences: 447634 from 183028 rows


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

In [ ]:

print(ask("What are the neurological symptoms of hyperthyroidism?"))
print(ask("Explain the difference between type 1 and type 2 diabetes.", style="short"))

In hyperthyroidism, neurologic symptoms include tremors of the hands, exophthalmos, lid lag, stare, and muscle weakness.As in this patient, symptoms and signs of hyperthyroxism may be subtle at initial presentation.> Electrocardiographic
Typeabetes mellitus, unlike type 1, does not have an autoimmune basis. It rules out diabetes mellitus type or type 2.


In [ ]:
print(ask("i have cold, suggest medicine for cold"))

In general, a "cold" lasting longer than 10 to 14 days with fever and facial pain is suggestive of rhinosinusitis. First generation antihistaminics possess anticholinergic propey and therefore are effective for common cold.Patients who have cold, suggest


In [ ]:
print(ask("how is blood pressure measured?"))

More points about measuring blood pressure The standard technique for blood pressure measurement is the auscultatory technique.The actual blood pressure can be determined by direct intraarterial measurement.


In [ ]:
print(ask("What are the typical treatment options for Type 2 Diabetes?"))

Only drugs approved for treatment of type diabetes are insulin and pramlintide. Insulin is the only treatment of Type DM.


In [ ]:
print(ask("how do i self manage Chronic back pain?"))

In patients with chronic low back pain (CLBP), bed rest should not exceed days. There are various modalities of treatment which are tried for chronic pain:- i) Pharmacotherapy:- The various drugs (mentioned above) are used for chronic Pain syndrome.


In [ ]:
print(ask("what is the treatment for hypertension?"))

Diet modification and drugsREF: Harrison 17th ed chapter 241"The mainstay of treatment of essential hypertension is lifestyle, diet modification and Drugs Management Surgical resection of the tumor is the treatment of choice and usually cures the hypertension.
